In [4]:
%pip install psycopg2-binary
%pip install pandas
%pip install numpy
%pip install scipy
%pip install ipywidgets
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


----
# Metadata Wide Association Study (MWAS)

PICKLE EVERYTHING AND READ IN LATER

In [5]:
# import required libraries
import psycopg2
import pandas as pd
import numpy as np
import scipy.stats as stats
import ipywidgets as widgets
import requests
import warnings
import io
import csv
import pickle
import time
import boto3

warnings.filterwarnings('ignore') # FIX THIS LATERE TO NOT IGNORE WARNINGS

total_start_time = time.time()

# database connection details
host = "serratus-aurora-20210406.cluster-ro-ccz9y6yshbls.us-east-1.rds.amazonaws.com"
database = "summary"
user = "public_reader"
password = "serratus"

# ensure connection is valid then close
try:
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    conn.close()
    print("Successfully connected to Serratus 🗻")
except:
    print("Error conecting to database.")

Successfully connected to Serratus 🗻


In [6]:
'''
rfamily database contains the number of reads of the virus for each run and the corresponding family_group

srarun database contains the number of spots for the run and the corresponding bio_sample, sample, bio_project, scientific_name

'''

# rfam_q = ''' SELECT run_id, family_group, percent_identity, n_reads
#                 FROM rfamily
#             '''

# srarun_q = ''' SELECT run, bio_sample, sample, bio_project, scientific_name, spots
#                 FROM srarun
#             '''

# with psycopg2.connect(host=host, database=database, user=user, password=password) as conn:
#     rfam_df = pd.read_sql(rfam_q, conn)
#     srarun_df = pd.read_sql(srarun_q, conn)

# # # --------------------- PICKLE ---------------------
# rfam_df.to_pickle('pickles/rfam_df.pickle')
# srarun_df.to_pickle('pickles/srarun_df.pickle')

# # --------------------- UNPICKLE ---------------------

with open('pickles/rfam_df.pickle', 'rb') as f:
    rfam_df = pickle.load(f)
    
with open('pickles/srarun_df.pickle', 'rb') as f:
    srarun_df = pickle.load(f)

In [8]:
# for i in {0..16}
# do
#     cp rfam_df.pickle "rfam_df_$i.pickle"
#     cp srarun_df.pickle "srarun_df_$i.pickle"
#     cp srarun_bioprojects_map.pickle "srarun_bioprojects_map_$i.pickle"
#     cp srarun_biosamples_map.pickle "srarun_biosamples_map_$i.pickle"
# done
# run to copy files to allow reading from multiple processes

In [9]:
# store runs that are in rfam but not in srarun?

In [10]:
num_zero_spots = srarun_df[srarun_df['spots'] == 0].shape[0]
print(f"Number of runs with 0 spots: {num_zero_spots}")
# replace 0 spots with 1000000
srarun_df['spots'] = srarun_df['spots'].replace(0, 1000000)

Number of runs with 0 spots: 0


In [11]:
# to be run on each family group (line in file)
family = 'Totiviridae-10'

# subset rfam_df by the family group
family_df = rfam_df[rfam_df['family_group'] == family]
family_df = pd.merge(family_df[['run_id', 'n_reads']], srarun_df[['run', 'bio_sample', 'spots']], left_on='run_id', right_on='run')
family_df

,run_id,n_reads,run,bio_sample,spots
0,SRR12369764,6,SRR12369764,SAMN15702630,95455823
1,SRR12369805,4,SRR12369805,SAMN15702699,50136340
2,SRR12362004,19,SRR12362004,SAMN15689828,43805632
3,SRR12214566,1,SRR12214566,SAMN15524028,62188730
4,SRR12333764,2,SRR12333764,SAMN15654926,43671791
...,...,...,...,...,...
76448,SRR9995119,56,SRR9995119,SAMN12598752,152058019
76449,SRR9995126,147,SRR9995126,SAMN12496165,38931600
76450,SRR9995127,59,SRR9995127,SAMN12496164,37833782
76451,SRR9995128,94,SRR9995128,SAMN12496163,42709834


In [12]:
# # mapping is done from biosamples rather than runs since runs have no metadata
# #   - spots/n_reads are averaged over all runs for the biosample
# #   - more importantly there is not always a run for each biosample 
# #   - alternatively runs could be used and biosamples that are missing runs are excluded
# srarun_biosamples_map = {}
# def srarun_biosamples_mapper(row):
#     srarun_biosamples_map[row['bio_sample']] = 0

# # use the map to apply a default n_read count to biosamples that have been analyzed by serratus
# start_time = time.time()
# srarun_df.apply(srarun_biosamples_mapper, axis=1)
# end_time = time.time()
# print(f"Time to create biosamples map: {end_time - start_time}")

# # create a map of biosamples to bioprojects
# srarun_bioprojects_map = {}
# def srarun_bioprojects_mapper(row):
#     srarun_bioprojects_map[row['bio_sample']] = row['bio_project']
    
# start_time = time.time()
# srarun_df.apply(srarun_bioprojects_mapper, axis=1)
# end_time = time.time()
# print(f"Time to create bioprojects map: {end_time - start_time}")


# # --------------------- PICKLE ---------------------
# # pickle the srarun_biosamples_map and srarun_bioprojects_map and save in the pickle folder
# #   - reduce running times by not having to create the above maps across all instances
# with open(f'pickles/srarun_biosamples_map.pickle', 'wb') as handle:
#     pickle.dump(srarun_biosamples_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(f'pickles/srarun_bioprojects_map.pickle', 'wb') as handle:
#     pickle.dump(srarun_bioprojects_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

# --------------------- UNPICKLE ---------------------
# retrieve the pickled dictionaries for srarun_biosamples_map and srarun_bioprojects_map
with open(f'pickles/srarun_biosamples_map.pickle', 'rb') as handle:
    srarun_biosamples_map = pickle.load(handle)

with open(f'pickles/srarun_bioprojects_map.pickle', 'rb') as handle:
    srarun_bioprojects_map = pickle.load(handle)


In [15]:

srarun_df.columns

Index(['run', 'bio_sample', 'sample', 'bio_project', 'scientific_name',
       'spots'],
      dtype='object')

In [29]:
# mapping also made for the family_df
# mapping for family_df stores the actual n_reads/spots value for each biosample
#   - if there is more than 1 value (multiple runs) then add to the list and take avg later
family_biosamples_n_reads = {}
family_biosample_spots = {}
# family_biosample_runs = {} # TO BE USED IF MAPPING IS DONE FROM RUNS
def family_biosamples_mapper(row):
    if row['bio_sample'] in family_biosamples_n_reads:
        family_biosamples_n_reads[row['bio_sample']].append(row['n_reads'])
        family_biosample_spots[row['bio_sample']].append(row['spots'])
        # family_biosample_runs[row['bio_sample']].append(row['run_id']) # TO BE USED IF MAPPING IS DONE FROM RUNS
    else:
        family_biosamples_n_reads[row['bio_sample']] = [row['n_reads']]
        family_biosample_spots[row['bio_sample']] = [row['spots']]
        # family_biosample_runs[row['bio_sample']] = [row['run_id']] # TO BE USED IF MAPPING IS DONE FROM RUNS

    
def family_bioproject_mapper(row):
    try:
        return srarun_bioprojects_map[row['bio_sample']]
    except:
        return np.nan

start_time = time.time()
family_df.apply(family_biosamples_mapper, axis=1)
end_time = time.time()
print(f"Time to map reads/spots: {end_time - start_time}")

start_time = time.time()
for biosample in family_biosamples_n_reads:
    family_biosamples_n_reads[biosample] = np.mean(family_biosamples_n_reads[biosample])
    family_biosample_spots[biosample] = np.mean(family_biosample_spots[biosample])
end_time = time.time()
print(f"Time to average reads/spots: {end_time - start_time}")


start_time = time.time()
family_df['bio_project'] = family_df.apply(family_bioproject_mapper, axis=1)
end_time = time.time()
print(f"Time to map bioprojects: {end_time - start_time}")
# NO LONGER NEED FAMILY_DF

Time to map reads/spots: 1.265510082244873
Time to average reads/spots: 0.8243417739868164
Time to map bioprojects: 0.4134852886199951


In [30]:
# set up variables for API call
base_url = 'https://serratus-biosamples.s3.us-east-1.amazonaws.com/bioprojects_csv/'
headers = {'Host': 'serratus-biosamples.s3.amazonaws.com'}

s3 = boto3.resource('s3')
bucket = s3.Bucket('serratus-biosamples')
# LOG THIS SHIT

In [31]:
# --------------- HELPER FUNCTIONS ---------------
# helper functions to use mappings to create n_reads and spots columns in bioproject_df
def get_n_reads(row):
    # try to get the n_reads from family_biosamples_n_reads, then from srarun_biosamples_map, then default to np.nan
    #   - tries to get exact n_reads from serratus
    #   - if run in serratus but no n_reads, then set to 0 (from srarun_biosamples_map)
    #   - if not run in serratus, then set to np.nan
    try:
        return family_biosamples_n_reads[row['biosample_id']]
    except:
        try:
            return srarun_biosamples_map[row['biosample_id']]
        except:
            return np.nan

def get_n_spots(row):
    # try to get the n_spots from family_biosample_spots, otherwise default to 0 
    ##### CHECK THIS ##### will have divide by 0 error if the mapping is wrong
    try:
        return family_biosample_spots[row['biosample_id']]
    except:
        return 0

def get_log_fold_change(true, false):
    # calculate the log fold change of true with respect to false
    #   - if true and false is 0, then return 0

    if true == 0 and false == 0:
        return 0
    elif true == 0:
        return -np.inf
    elif false == 0:
        return np.inf
    else:
        return np.log2(true / false)
    

In [32]:
# get the bioprojects for the family and remove None
bioprojects = list(family_df['bio_project'].unique())
bioprojects.remove(None)
# a number of bioprojects are not stored in srarun_df (26094) 
#   - the biosamples for these bioprojects cannot be analzed unless other biosamples 
#       in the bioproject are in the same family_group and have the project listed in srarun_df

# iterate through the projects for the family
for bioproject_id in bioprojects:
    # --------------- SET UP DF FOR MWAS ---------------
    # get the csv file for the bioproject
    url = base_url + bioproject_id + '.csv'
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f'Error: {response.status_code} for {bioproject_id}') # LOG THIS
        continue
    # read the csv file into a dataframe
    bp_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), quoting=csv.QUOTE_NONE)

    # skip if there are two rows or less
    num_rows = bp_df.shape[0]
    if num_rows <= 2:
        # LOG HERE
        print(f"Bioproject {bioproject_id} has {num_rows} rows")
        continue

    # add a dummy column to the dataframe to test agregation
    bp_df['dummy'] = bp_df.apply(lambda row: 1 if row['dev_stage'] == 'mixed propagative' else 0, axis=1)
    bp_df['dummy2'] = np.array([1, 2, 1, 1, 2, 2])

    # add the n_reads and n_spots columns to the dataframe
    start_time = time.time()
    bp_df['n_reads'] = bp_df.apply(get_n_reads, axis=1)
    bp_df['n_spots'] = bp_df.apply(get_n_spots, axis=1)
    end_time = time.time()
    print(f"Time to add n_reads and n_spots: {end_time - start_time}")

    # add the rpm column to the dataframe
    start_time = time.time()
    bp_df['rpm'] = bp_df.apply(lambda row: row['n_reads'] / row['n_spots'] * 1000000 if row['n_spots'] != 0 else 0, axis=1)
    end_time = time.time()
    print(f"Time to add rpm: {end_time - start_time}")
    

    # --------------- SELECTING COLUMNS TO TEST ---------------
    remove_cols = {'biosample_id','n_reads','n_spots','rpm'}
    target_cols = list(set(bp_df.columns) - remove_cols)

    metadata_counts = {}
    num_rows = bp_df.shape[0]

    start_time = time.time()
    # store the value counts for each column
    for col in target_cols:
        counts = bp_df[col].value_counts()
        n = len(counts)
        # skip if there is only 1 value
        if n == 1:
            continue
        # skip if all values are unique
        if n == num_rows:
            continue
        metadata_counts[col] = counts.to_dict()

    end_time = time.time()
    print(f"Time to get value counts: {end_time - start_time}")


    existing_runs = list()
    target_col_runs = {}
    for target_col, value_counts in metadata_counts.items():
        for target_term, count in value_counts.items():
            # skip if there is only 1 value
            if count == 1:
                continue

            # get the biosample_ids for the target term
            target_term_biosamples = list(bp_df[bp_df[target_col] == target_term]['biosample_id'])
            # check if the same biosamples are already stored and aggregate the columns
            target_col_name = f'{target_col}\t{target_term}'

            if target_term_biosamples in existing_runs:
                pass
                existing_key = list(target_col_runs.keys())[list(target_col_runs.values()).index(target_term_biosamples)]
                # print(f'merge: {existing_key} and {target_col_name}')
                target_col_name = f'{existing_key}\r{target_col_name}'
                # update the dictionary with the new name
                target_col_runs[target_col_name] = target_col_runs.pop(existing_key)
            else:
                existing_runs.append(target_term_biosamples)
                target_col_runs[target_col_name] = target_term_biosamples


    # --------------- RUN TTESTS ---------------
    # set up cols for the output df
    output_cols = ['bioproject_id', 'metadata_field', 'metadata_value', 'num_true', 'num_false', 'mean_rpm_true', 'mean_rpm_false', 'sd_rpm_true', 'sd_rpm_false', 'fold_change', 't_statistic', 'p_value']
    output_dict = {col: [] for col in output_cols}

    start_time = time.time()
    for target_col, biosamples in target_col_runs.items():      
        num_true = len(biosamples)
        num_false = num_rows - num_true
        # get the rpm values for the target and remainng biosamples
        true_rpm = bp_df[bp_df['biosample_id'].isin(biosamples)]['rpm']
        false_rpm = bp_df[~bp_df['biosample_id'].isin(biosamples)]['rpm']
        
        # calculate desecriptive stats
        # NON CORRECTED VALUES
        mean_rpm_true = np.nanmean(true_rpm)
        mean_rpm_false = np.nanmean(false_rpm)
        sd_rpm_true = np.nanstd(true_rpm)
        sd_rpm_false = np.nanstd(false_rpm)

        # calculate fold change and check if any values are nan
        fold_change = get_log_fold_change(mean_rpm_true, mean_rpm_false)
        # calculate t-statistic and p-value
        t_statistic, p_value = stats.ttest_ind_from_stats(mean1=mean_rpm_true, std1=sd_rpm_true, nobs1=num_true, mean2=mean_rpm_false, std2=sd_rpm_false, nobs2=num_false, equal_var=False)

        # extract metadata_field (column names) and metadata_value (column values)
        metadata_tmp = target_col.split('\r') # pairs of metadata_field and metadata_value for aggregated columns
        # metadata_field = '\t'.join(metadata_tmp[::2])
        # metadata_value = '\t'.join(metadata_tmp[1::2])
        metadata_field = '\t'.join(pair.split('\t')[0] for pair in metadata_tmp)
        metadata_value = '\t'.join(pair.split('\t')[1] for pair in metadata_tmp)


        # add values to output dict
        output_dict['bioproject_id'].append(bioproject_id)
        output_dict['metadata_field'].append(metadata_field)
        output_dict['metadata_value'].append(metadata_value)
        output_dict['num_true'].append(num_true)
        output_dict['num_false'].append(num_false)
        output_dict['mean_rpm_true'].append(mean_rpm_true)
        output_dict['mean_rpm_false'].append(mean_rpm_false)
        output_dict['sd_rpm_true'].append(sd_rpm_true)
        output_dict['sd_rpm_false'].append(sd_rpm_false)
        output_dict['fold_change'].append(fold_change)
        output_dict['t_statistic'].append(t_statistic)
        output_dict['p_value'].append(p_value)

    # create the output df and sort by p_value
    mwas_df = pd.DataFrame(output_dict)
    mwas_df = mwas_df.sort_values(by='p_value')

    # log if there are significant results
    significant = mwas_df[mwas_df['p_value'] < 0.05].shape[0]
    if significant > 0:
        print(f"Bioproject {bioproject_id} has {significant} significant results")

    # write the mwas_df to a csv file on s3
    key = f's3://serratus-biosamples/mwas/{family}/{bioproject_id}.csv'
    mwas_df.to_csv(key, index=False)
    # display(mwas_df)
    break

    end_time = time.time()
    print(f"Time to run t-tests: {end_time - start_time}")
    break


# total_end_time = time.time()
# print(f"Total time: {total_end_time - total_start_time}")

Error: 404 for PRJNA650304
Error: 404 for PRJNA650306
Error: 404 for PRJNA649960
Error: 404 for PRJNA646066
Error: 404 for PRJNA648996
Bioproject PRJNA618938 has 1 rows
Bioproject PRJNA618909 has 1 rows
Error: 404 for PRJDA48395
Error: 404 for PRJDA67119
Error: 404 for PRJDA74549
Error: 404 for PRJDB2660
Error: 404 for PRJDB1757
Error: 404 for PRJDB2676
Error: 404 for PRJDB3324
Error: 404 for PRJDB2026
Error: 404 for PRJDB1881
Error: 404 for PRJDB2433
Error: 404 for PRJDB2721
Error: 404 for PRJDB3309
Error: 404 for PRJDB3393
Time to add n_reads and n_spots: 0.0009379386901855469
Time to add rpm: 0.0005319118499755859
Time to get value counts: 0.0043182373046875
Bioproject PRJDB3458 has 1 significant results


In [34]:
target_col_runs

{'last_update\t2015-04-01T19:36:01.000\rpublication_date\t2015-04-02T08:36:01.000': ['SAMD00025070',
  'SAMD00025069',
  'SAMD00025065'],
 'dummy2\t1': ['SAMD00025070', 'SAMD00025065', 'SAMD00025066'],
 'dummy2\t2': ['SAMD00025069', 'SAMD00025067', 'SAMD00025068'],
 'publication_date\t2015-04-02T00:00:00.000\rsubmission_date\t2015-09-16T16:57:05.000': ['SAMD00025066',
  'SAMD00025067',
  'SAMD00025068'],
 'host\tPinus thunbergii': ['SAMD00025069', 'SAMD00025068'],
 'dummy\t1\rdev_stage\tmixed propagative': ['SAMD00025070',
  'SAMD00025069',
  'SAMD00025065',
  'SAMD00025066',
  'SAMD00025068']}

In [37]:
srarun_biosamples_map

{'SAMN14170104': 0,
 'SAMN14170103': 0,
 'SAMN14170102': 0,
 'SAMN14170101': 0,
 'SAMN14170100': 0,
 'SAMN14170099': 0,
 'SAMN14170098': 0,
 'SAMN14170097': 0,
 'SAMN14170353': 0,
 'SAMN14170352': 0,
 'SAMN14170351': 0,
 'SAMN14170350': 0,
 'SAMN14170347': 0,
 'SAMN14170346': 0,
 'SAMN14170345': 0,
 'SAMN14170344': 0,
 'SAMN14170349': 0,
 'SAMN14170348': 0,
 'SAMN14170343': 0,
 'SAMN14170342': 0,
 'SAMN14170341': 0,
 'SAMN14170340': 0,
 'SAMN14170339': 0,
 'SAMN14170338': 0,
 'SAMN14170337': 0,
 'SAMN14170336': 0,
 'SAMN14170335': 0,
 'SAMN14170375': 0,
 'SAMN14170374': 0,
 'SAMN14170373': 0,
 'SAMN14170372': 0,
 'SAMN14170371': 0,
 'SAMN14170370': 0,
 'SAMN14170369': 0,
 'SAMN14170368': 0,
 'SAMN14170367': 0,
 'SAMN14172787': 0,
 'SAMN14172786': 0,
 'SAMN14172785': 0,
 'SAMN14172784': 0,
 'SAMN14172783': 0,
 'SAMN14172782': 0,
 'SAMN14172781': 0,
 'SAMN14172742': 0,
 'SAMN14172740': 0,
 'SAMN14172739': 0,
 'SAMN14172738': 0,
 'SAMN14172737': 0,
 'SAMN14172736': 0,
 'SAMN14172735': 0,


In [35]:
bp_df

,biosample_id,submission_date,last_update,publication_date,access,BioSample_db,title,organism_taxonomy_id,organism_taxonomy_name,Name,...,status,when,paragraph,host,SRA_db,dummy,dummy2,n_reads,n_spots,rpm
0,SAMD00025070,2015-04-01T12:05:10.040,2015-04-01T19:36:01.000,2015-04-02T08:36:01.000,public,SAMD00025070,B. xylophilus mixed-stage,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-04-01T12:05:10.040,B.xylophilus mixed-stage on B. cinerea,NaN,NaN,1,1,0.0,0.0,0.000000
1,SAMD00025069,2015-04-01T12:05:10.163,2015-04-01T19:36:01.000,2015-04-02T08:36:01.000,public,SAMD00025069,B.xylophilus infected Sep,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-04-01T12:05:10.163,B.xylophilus infected in September,Pinus thunbergii,NaN,1,2,0.0,0.0,0.000000
2,SAMD00025065,2015-04-01T12:05:10.230,2015-04-01T19:36:01.000,2015-04-02T08:36:01.000,public,SAMD00025065,B.xylophilus cont Aug,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-04-01T12:05:10.230,B.xylophilus control in August,NaN,NaN,1,1,0.0,0.0,0.000000
3,SAMD00025066,2015-09-16T16:57:05.000,2023-07-10T23:11:41.430,2015-04-02T00:00:00.000,public,SAMD00025066,B.xylophilus cont Sep,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-09-16T16:57:05.049,B.xylophilus control in September,NaN,DRS033339,1,1,0.0,0.0,0.000000
4,SAMD00025067,2015-09-16T16:57:05.000,2023-07-10T23:11:41.670,2015-04-02T00:00:00.000,public,SAMD00025067,B. xylophilus D4 stage,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-09-16T16:57:05.169,B.xylophilus D4 stage,Monochamus alternatus,DRS022442,0,2,6.0,13172530.5,0.455493
5,SAMD00025068,2015-09-16T16:57:05.000,2023-07-10T23:11:41.480,2015-04-02T00:00:00.000,public,SAMD00025068,B.xylophilus infected Aug,6326,Bursaphelenchus xylophilus,Faculty of Medicine Parasitology,...,live,2015-09-16T16:57:05.272,B.xylophilus infected in August,Pinus thunbergii,DRS171956,1,2,1.0,11061749.0,0.090402
